In [1]:
import clsDatabase as db
import pandas as pd
#pip install pandas 필요
import datetime
from sklearn import utils
#pip install scikit-learn, scipy 필요
import tensorflow as tf
import numpy as np
#pip install numpy


In [2]:
#Database에서 자료 가져오기
query = 'SELECT candle_date_time_utc, candle_date_time_kst, opening_price, high_price, low_price, '
query += ' trade_price, timestamp, candle_acc_trade_price, candle_acc_trade_volume, unit,'
query += ' gap, [check] '
query += ' FROM TradingTest'

table = db.SelectQuery(query)

db.ConnectionClose()

In [3]:
#데이터 형변환
for rows in table :
     for colFieldName in rows :
         if type(rows[colFieldName]) == datetime.datetime :
            rows[colFieldName] = rows[colFieldName].strftime('%Y%m%d%H%M')
         elif rows[colFieldName] == 'Up':
             rows[colFieldName] = 1
         elif rows[colFieldName] == 'Down' : 
             rows[colFieldName] = 0


data = pd.DataFrame(table)
data = data.convert_dtypes()

data.dtypes

candle_date_time_utc        string
candle_date_time_kst        string
opening_price                Int64
high_price                   Int64
low_price                    Int64
trade_price                  Int64
timestamp                    Int64
candle_acc_trade_price     Float64
candle_acc_trade_volume    Float64
unit                         Int64
gap                          Int64
check                       object
dtype: object

In [4]:
trainRate = 0.8 #학습용 데이터 80%, 테스트용 20%
resultColName = 'check'
data = utils.shuffle(data)

trainSize = int(data.shape[0] * trainRate)

trainData = data.iloc[:trainSize, :]
trainLearningData = trainData.loc[:, trainData.columns != resultColName]
trainResultData = trainData.loc[:, resultColName]

testData = data.iloc[trainSize:, :]
testLearningData = testData.loc[:, testData.columns != resultColName]
testResultData = testData.loc[:, resultColName]

In [5]:
trainLearningData = np.asarray(trainLearningData).astype(np.float32)
trainResultData = np.asarray(trainResultData).astype(np.float32)
testLearningData = np.asarray(testLearningData).astype(np.float32)
testResultData = np.asarray(testResultData).astype(np.float32)


In [6]:
Input = tf.keras.layers.Input(shape=trainLearningData.shape[0], name="Input")
#Flatten = tf.keras.layers.Flatten(input_shape=int(trainLearningData.shape[0]), name="Flatten")
Dense1 = tf.keras.layers.Dense(64,  activation='relu', name="Dense1")(Input)
Dense2 = tf.keras.layers.Dense(256, activation='relu', name="Dense2")(Dense1)
Dense3 = tf.keras.layers.Dense(64,  activation='relu', name="Dense3")(Dense2)
Dense4 = tf.keras.layers.Dense(16,  activation='relu', name="Dense4")(Dense3)
Output = tf.keras.layers.Dense(1,   activation='softmax', name="Output")(Dense4)

model = tf.keras.Model(inputs=Input, outputs=Output)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 160)]             0         
                                                                 
 Dense1 (Dense)              (None, 64)                10304     
                                                                 
 Dense2 (Dense)              (None, 256)               16640     
                                                                 
 Dense3 (Dense)              (None, 64)                16448     
                                                                 
 Dense4 (Dense)              (None, 16)                1040      
                                                                 
 Output (Dense)              (None, 1)                 17        
                                                                 
Total params: 44,449
Trainable params: 44,449
Non-trainable p

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics='accuracy')
model.fit(trainLearningData, trainResultData, epochs=3, validation_data=(testLearningData, testResultData))

Epoch 1/3


ValueError: in user code:

    File "C:\Users\smtvision\anaconda3\envs\DeepLearning\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\smtvision\anaconda3\envs\DeepLearning\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\smtvision\anaconda3\envs\DeepLearning\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\smtvision\anaconda3\envs\DeepLearning\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\smtvision\anaconda3\envs\DeepLearning\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\smtvision\anaconda3\envs\DeepLearning\lib\site-p

In [ ]:
# del query
# del value, rows, table
# del trainRate, resultFieldname, data, trainSize, trainData, testData
# del Input, Dense1, Dense2, Dense3, Dense4, Output